# Developing multiple-shooting in Sun-B1 dynamics

2022.11.23

In [1]:
using DifferentialEquations
using Plots
using LinearAlgebra
import ForwardDiff
import DiffResults
using AstrodynamicsBase
# import joptimise
using Printf
using JSON


In [2]:
include("../../julia-r3bp/R3BP/src/R3BP.jl")
include("../src/SailorMoon.jl")   # relative path to main file of module
param3b = SailorMoon.dynamics_parameters()

┌ Warning: Replacing docs for `Main.SailorMoon.integrate_rk4 :: Union{Tuple{ODEProblem, Real}, Tuple{ODEProblem, Real, Union{Nothing, Vector{T} where T}}, Tuple{ODEProblem, Real, Union{Nothing, Vector{T} where T}, Bool}, Tuple{ODEProblem, Real, Union{Nothing, Vector{T} where T}, Bool, Bool}}` in module `Main.SailorMoon`
└ @ Base.Docs docs\Docs.jl:240


Main.SailorMoon.dynamics_params(0.987849414390376, 0.01215058560962404, 328900.5598102475, 384748.32292972936, 375700.3437894195, 388.8212386592645, -0.9251999994040079, 0.9251999994040079, 0.07480000059599223, 0.01709689063726318, 7.601281331451572)

In [3]:
plotly()

┌ Warning: For saving to png with the `Plotly` backend `PlotlyBase` and `PlotlyKaleido` need to be installed.
│   err =
│    ArgumentError: Package PlotlyBase not found in current path:
│    - Run `import Pkg; Pkg.add("PlotlyBase")` to install the PlotlyBase package.
│    
└ @ Plots C:\Users\yujit\.julia\packages\Plots\nuwp4\src\backends.jl:545


Plots.PlotlyBackend()

In [4]:
function plot_circle(radius, x, y, n=100)
    circle = zeros(2,n)
    thetas = LinRange(0.0, 2π, n)
    for i = 1:n
        circle[1,i] = radius*cos(thetas[i]) + x
        circle[2,i] = radius*sin(thetas[i]) + y
    end
    return circle
end

plot_circle (generic function with 2 methods)

In [5]:
### PARAMETERS ###################################
# csv file to load the initial solution
filename = "../run/grid_search1129.csv"
# dv_dir function corresponding to the csv file 
dir_func = SailorMoon.dv_no_thrust 

# 3body parameter
param3b = SailorMoon.dynamics_parameters()

n_arc = 5

# run minimizer with IPOPT
ip_options = Dict(
    "max_iter" => 2500,   # 1500 ~ 2500
    "tol" => 1e-6,
    "output_file" => "ELET_ipopt.out"
)
##################################################

Dict{String, Any} with 3 entries:
  "tol"         => 1.0e-6
  "output_file" => "ELET_ipopt.out"
  "max_iter"    => 2500

In [ ]:
if dir_func == SailorMoon.dv_no_thrust
    τ_ig = 0.0
else 
    τ_ig = 1.0
end

# load initial guess
df = DataFrame(CSV.File(filename))

# maybe want to use "for row in eachrow(df)" to automate the process...? 
row = df[2,:]

x0, lx, ux = SailorMoon.make_ig_bounds(row, τ_ig, n_arc)

# res = SailorMoon.multishoot_trajectory(x0, dir_func, n_arc)
# print("size of res: ", size(res))

fitness!, ng, lg, ug, eval_sft = SailorMoon.get_fitness(n_arc, dir_func, x0)

# checking if the initial guess is good enough
res = eval_sft(x0)
# println("ub - x0: ", ux - x0)
# println("x0 - lb: ", x0 - lx)
# println("x0: ", x0)
# println("residual (needs to be 0): ", res)

# make sure the initial guess is inbetween ub & lb
vec = vcat(ux-x0, x0 - lx)
if any(vec .< 0.0)
    error("Error: (At least one element of) initial guess is infinging the defined ub/lb.") 
end

xopt, fopt, Info = joptimise.minimize(fitness!, x0, ng;
    lx=lx, ux=ux, lg=lg, ug=ug, solver="ipopt",
    options=ip_options, outputfile=true, 
)  # derivatives=joptimise.UserDeriv());  # to use AD, need this additional parameter...

println(Info)